In [ ]:
%pip install databricks-vectorsearch

In [ ]:
import time

from databricks.vector_search.client import VectorSearchClient

from common import utils
from common import vector_search_config

client = VectorSearchClient()
poll_secs = 5


def _get_vector_search_endpoint(name: str):
    try:
        return client.get_endpoint(name=name)
    except Exception:
        return None


while True:
    ep = _get_vector_search_endpoint(vector_search_config.ENDPOINT_NAME)
    if ep:
        state = ep["endpoint_status"]["state"]
        utils.logger().info("%s state: %s", vector_search_config.ENDPOINT_NAME, state)
        if state == "ONLINE":
            break
        if state == "PROVISIONING":
            time.sleep(poll_secs)
            continue
        raise ValueError(f"unknown state: {state}")

    utils.logger().info("creating vector search endpoint: %s", vector_search_config.ENDPOINT_NAME)
    client.create_endpoint_and_wait(vector_search_config.ENDPOINT_NAME, endpoint_type="STANDARD")
    break